<a href="https://colab.research.google.com/github/huijunam/CJ_AI_hackathon/blob/eunhy/addGIPHY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import output

In [2]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1
!pip install ipyplot

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

output.clear()

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
import datetime

In [4]:
!git clone https://github.com/huijunam/CJ_AI_hackathon.git

Cloning into 'CJ_AI_hackathon'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 88 (delta 7), reused 1 (delta 0), pack-reused 69
Receiving objects: 100% (88/88), 4.18 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import ipyplot

chatbot_data_1 = pd.read_excel('/content/CJ_AI_hackathon/data/8f3202096f1387c5.xlsx')
chatbot_data_2 = pd.read_excel('/content/CJ_AI_hackathon/data/ad8cb80ea8623bbe.xlsx')


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


In [6]:
chatbot_data_1= chatbot_data_1.iloc[:,:2]
chatbot_data_2 = chatbot_data_2.iloc[1:, :3]
chatbot_data_2 = chatbot_data_2.iloc[:,1:]

# 90 이하인 카테고리 추출
value_counts = chatbot_data_2['Unnamed: 2'].value_counts()
categories_to_remove = value_counts[value_counts <= 90].index

# 해당 카테고리를 포함하지 않는 행만 선택
chatbot_data_2 = chatbot_data_2[~chatbot_data_2['Unnamed: 2'].isin(categories_to_remove)]

chatbot_data_2 = chatbot_data_2.dropna(how='any')

#최종 데이터
df_data = pd.concat([chatbot_data_1,chatbot_data_2], ignore_index=True)

In [7]:
target_classes ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

for cls in target_classes:
    df_data.loc[(df_data['Emotion'] == cls), 'Emotion'] = target_classes[cls]

data_list =[]
# ['sentence', 'class'] 형태로 변환
for q, label in zip(df_data['Sentence'], df_data['Emotion']):
    data =[]
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [8]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [9]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

device = torch.device(device_type)


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
def get_kobert_model(model_path, vocab_file, ctx=device):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [12]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(target_classes),   # 클래스 현재 7개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/CJ/base_model_koBERT_epoch3_.pt'))

<All keys matched successfully>

In [15]:
!pip install konlpy

from konlpy.tag import Okt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 36.2 MB/s eta 0:00:00


In [16]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a) # 수정한 부분
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=bb0e8601499c1ec1e79f905daa54a6178527413854c85f7ad404e4285c4d49e7
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=7ea14b9b05fa2b4893d7a28bfc8999aa3a9578fc744f90f03ce6677a5b56c24d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [19]:
from keybert import KeyBERT
kw_model = KeyBERT(model)

In [23]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('혼자', 0.5736), ('힘들', 0.2377), ('많아', 0.168), ('때도', 0.1018), ('사는', 0.0629)]

In [22]:
text = '혼자 사는 게 힘들 때도 많아.'

In [21]:
result_text = ''
text = '혼자 사는 게 힘들 때도 많아.'
for sentence in kiwi.analyze(text):
    nouns = []
    for token in sentence[0]:
        if token.tag.startswith('NN'):
            nouns.append(token.form)
    if nouns:
        result_text = ' '.join(nouns)
print(result_text)

NameError: ignored

In [24]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [26]:
print('OKT 명사 추출 :',okt.pos(text))
print('Hannanum 명사 추출 :',hannanum.pos(text))
print('Kkma 명사 추출 :',kkma.pos(text))
print('Komoran 명사 추출 :',komoran.pos(text))
print("=="*30)

OKT 명사 추출 : [('혼자', 'Noun'), ('사는', 'Verb'), ('게', 'Noun'), ('힘들', 'Adjective'), ('때', 'Noun'), ('도', 'Josa'), ('많아', 'Adjective'), ('.', 'Punctuation')]
Hannanum 명사 추출 : [('혼자', 'M'), ('사', 'P'), ('는', 'E'), ('것', 'N'), ('이', 'J'), ('힘들', 'P'), ('ㄹ', 'E'), ('때', 'N'), ('도', 'J'), ('많', 'P'), ('아', 'E'), ('.', 'S')]
Kkma 명사 추출 : [('혼자', 'NNG'), ('사', 'VV'), ('는', 'ETD'), ('것', 'NNB'), ('이', 'JKS'), ('힘들', 'VA'), ('ㄹ', 'ETD'), ('때', 'NNG'), ('도', 'JX'), ('많', 'VA'), ('아', 'ECD'), ('.', 'SF')]
Komoran 명사 추출 : [('혼자', 'NNG'), ('살', 'VV'), ('는', 'ETM'), ('게', 'NNG'), ('힘들', 'VA'), ('ㄹ', 'ETM'), ('때', 'NNG'), ('도', 'JX'), ('많', 'VA'), ('아', 'EF'), ('.', 'SF')]


In [31]:
pos_list = ['Noun','Verb','Adjective']
words_list = []
for word in okt.pos(text):
    if word[-1] in pos_list:
        words_list.append(word[0])
words_list


['혼자', '사는', '게', '힘들', '때', '많아']

In [33]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('혼자', 0.5736), ('힘들', 0.2377), ('많아', 0.168), ('때도', 0.1018), ('사는', 0.0629)]

In [ ]:
for x,y in keywords:
    if y >= 0.15:
        test_eval.append(x)

In [117]:
# 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

def predict_and_keword(predict_sentence):
  new_data =[[predict_sentence, '0']]
  # print(new_data)

  new_dataset = BERTDataset(new_data, 0, 1, tokenizer, vocab ,max_len, True, False)
  new_dataloader = torch.utils.data.DataLoader(new_dataset, batch_size = batch_size, num_workers =5)

  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(new_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    test_eval =[]
    for logit in out:
      test_eval.append(reverse[np.argmax(logit.detach().cpu().numpy())])

    # 키워드 추출: 명사, 동사, 형용사 추출하기
    keyword = [x for (x, y) in okt.pos(predict_sentence) if y in ['Verb', 'Noun', 'Adjective']]
    keywords = kw_model.extract_keywords(keyword, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
    #print('\nOKT 명사, 동사, 형용사 추출 :', keyword)
    for i in keywords:
        print(i)
        if len(i)==0:
            pass
        else:
            if i[0][1]  >= 0.5:
                test_eval.append(i[0][0])
    return test_eval

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [118]:
# 하이퍼 파라미터 설정
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
random_seed = 42

In [119]:
reverse =dict(map(reversed, target_classes.items()))

# 여기서부터 GIF입니다!

In [120]:
import json
import random
from urllib import parse, request
import re
from IPython.display import Image

url = "http://api.giphy.com/v1/gifs/search"

In [121]:
sentence = input("하고싶은 말을 입력해주세요 : ")
array = predict_and_keword(sentence)
print(array)
data_list = []
for item in array:
    params = parse.urlencode({
        "q": item,
        "api_key": "P9SzTTpADkMNu3XjLkzRyMxVGm3MyKDQ",
        "limit": "50"
        })

    try:
        with request.urlopen("".join((url, "?", params))) as response:
            data = json.loads(response.read())

        get_nums = set()

        while len(get_nums) < (len(data['data']))/5:
            get_nums.add(random.randint(0,len(data['data'])+1))

        for i in get_nums:
            #print(data['data'][i]['images']['downsized']['url'])
            data_list.append(data['data'][i]['images']['downsized']['url'])

    except:
        pass

get_nums = set()
while len(get_nums) != 18:
    get_nums.add(random.randint(0,len(data_list)-1))

data_gif = []
for i in get_nums:
    data_gif.append(data_list[i])


for idx, value in enumerate(data_gif):
    data_gif[idx] = re.sub('media+[0-9]', 'i', value)

ipyplot.plot_images(data_gif,img_width=200,show_url=False)

하고싶은 말을 입력해주세요 :  정말 보보경심 제작사가 아이유보다 더 이해 안 가.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/1 [00:00<?, ?it/s]

[('정말', 1.0)]
[('보보경심', 1.0)]
[('제작사', 1.0)]
[('아이유', 1.0)]
[]
[('이해', 1.0)]
[]
[]
['혐오', '정말', '보보경심', '제작사', '아이유', '이해']


gif를 위처럼 한 줄로 띄워서 보여주고 싶은데 아직 더 좋은 방법을 찾지 못했어요...